```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import CTCS, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *
from SWESimulators import DataAssimilationUtils as dautils

from SWESimulators import WindForcingEnsemble

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

# The Lorenz Attractor (Lorenz-63)

In [ ]:

class Lorenz63Ensemble:
        
    def __init__(self, cl_ctx, numParticles, \
                 observation_variance=np.sqrt(2), model_error=np.sqrt(2)*0.01, init_error=np.sqrt(2),\
                 T=10):

        self.T = T
        
        self.numParticles = numParticles
        self.particles = [None]*(self.numParticles + 1)
        
        self.obs_index = self.numParticles
                
        self.observation_variance = observation_variance
        self.model_error = model_error
        self.init_error = init_error
        
        # Use parameters from the PJvL-2010 paper
        self.dt = 0.01
        self.sigma = 10
        self.rho = 28
        self.beta = 8.0/3.0
        self.initValues = [1.508870, -1.531271, 25.46091]
        
        self.timestep = 0
        self.t = 0.0
        
        self.init()
        self.observations = []

    # ---------------------------------------
    # Override functions we don't need.
    # ---------------------------------------       
    def cleanUp(self):
        pass
        
    def setGridInfo(self, nx, ny, dx, dy, dt, boundaryConditions, eta=None, hu=None, hv=None, H=None):
        pass
    
    def setParameters(self, f=0, g=9.81, beta=0, r=0, wind=None):
        pass
    
    
    # ---------------------------------------
    # Needs to be abstract!
    # ---------------------------------------
    def init(self):
        self.max_timesteps = self.T+1
        for p in range(self.numParticles+1):
            self.particles[p] = np.zeros((3, self.max_timesteps))
            self.particles[p][:,0] = np.random.multivariate_normal(self.initValues, np.eye(3)*self.init_error )
    
    
    def observeParticles(self):
        positions = np.zeros((3,self.numParticles))
        for p in range(self.numParticles):
            positions[:,p] = self.particles[p][:,int(self.timestep)]
        return positions
        
    def observeTrueState(self):
        obs = np.random.multivariate_normal(self.particles[self.obs_index][:,int(self.timestep)], np.eye(3)*self.observation_variance)
        self.observations.append((obs,self.timestep))
        return obs
    
    def step(self, t):
        
        k_end = min(self.timestep + t, self.max_timesteps-1)
        for p in self.particles:
            for k in range(int(self.timestep), int(k_end)):
                model_error = np.random.multivariate_normal(np.zeros(3), np.eye(3)*self.model_error)
                p[0,k+1] = p[0,k] + self.dt*(self.sigma*(p[1,k] - p[0,k])) + model_error[0]
                p[1,k+1] = p[1,k] + self.dt*(p[0,k]*(self.rho - p[2,k]) - p[1,k]) + model_error[1]
                p[2,k+1] = p[2,k] + self.dt*(p[0,k]*p[1,k] - self.beta*p[2,k]) + model_error[2]
        self.timestep = k_end
        self.t = k_end*self.dt
                
    def getDistances(self, obs=None):
        return self.sim.drifters.getDistances()
       
    def resample(self, newSampleIndices, reinitialization_variance):
        self.sim.drifters.resample(newSampleIndices, reinitialization_variance)
                    
    def getEnsembleMean(self):
        mean = np.zeros_like(self.particles[0])
        for j in range(self.numParticles):
            mean += self.particles[j]
        mean = mean/self.numParticles
        return mean
    
    def _plot_index(self, i, mean):
        particle_number = 0
        
        t = range(0, self.max_timesteps)
        for p in self.particles:
            if particle_number != self.obs_index:
                plt.plot(t, p[i,:], color="0.8" )
            else:
                plt.plot(t,p[i,:], 'g')
            particle_number += 1
        plt.plot(t, mean[i, :], 'r' )
        for o in range(len(self.observations)):
            plt.plot(ensemble.observations[o][1], ensemble.observations[o][0][i], 'bo')
        
    
    def plotDistanceInfo(self, title=None, obs=None):
        mean = self.getEnsembleMean()
        fig = plt.figure(figsize=(12,8))
        plt.subplot(3,1,1)    
        self._plot_index(0, mean)
        plt.subplot(3,1,2)    
        self._plot_index(1, mean)
        plt.subplot(3,1,3)    
        self._plot_index(2, mean)
        
            
    def enforceBoundaryConditions(self):
        self.sim.drifters.enforceBoundaryConditions()
    
    ### Duplication of code
    def getDomainSizeX(self):
        pass
    def getDomainSizeY(self):
        pass
    def getObservationVariance(self):
        pass
    def getNumParticles(self):
        return self.numParticles
    


In [ ]:
T = 1000
ensemble = Lorenz63Ensemble(cl_ctx, 10, T=T, model_error=0, init_error=0.001)
ensemble.step(T-50)
#print ensemble.observeParticles()
print ensemble.observeTrueState()
ensemble.step(50)
ensemble.plotDistanceInfo()
